# 1. Chi tiết về API backend


Dưới đây là **phân tích và giải thích** chi tiết về mã nguồn `api_database_app.py` của bạn, cùng với gợi ý một số cải tiến nếu cần thiết:

## **Giới thiệu về các API**
Mã nguồn hiện tại cung cấp 3 API chính:
1. **`/get_video_data`** - Lấy thông tin video từ cơ sở dữ liệu.
2. **`/save_score`** - Lưu hoặc cập nhật điểm số cho một video dựa trên tiêu chí cụ thể.
3. **`/get_scores`** - Lấy danh sách các điểm số cho một video cụ thể.

---

## **Chi tiết về từng API**

### 1. **API: `/get_video_data`**
- **Phương thức**: `GET`
- **Tham số yêu cầu**: `url` (video URL)
- **Chức năng**:
  - Truy xuất thông tin của một video từ bảng `videos` trong cơ sở dữ liệu SQLite.
  - Trả về `url_video`, `transcript` và `criteria` nếu tìm thấy.
  - Trả về lỗi `404` nếu video không có trong cơ sở dữ liệu.

#### **Ví dụ sử dụng:**
```shell
GET http://localhost:3000/get_video_data?url=https://example.com/video1
```

**Response**:
```json
{
    "url_video": "https://example.com/video1",
    "transcript": "This is the transcript...",
    "criteria": "Fluency"
}
```

---

### 2. **API: `/save_score`**
- **Phương thức**: `POST`
- **Body yêu cầu (JSON)**:
  ```json
  {
    "url_video": "https://example.com/video1",
    "criteria": "Fluency",
    "score": 8,
    "note": "Good job!"
  }
  ```
- **Chức năng**:
  - Lưu điểm số cho một video dựa trên tiêu chí (criteria).
  - Nếu đã tồn tại bản ghi với `url_video` và `criteria`, thì sẽ cập nhật `score`, `note`, và `timestamp`.
  - Nếu chưa tồn tại, thì tạo mới một bản ghi.
  - Tạo bảng `scores` nếu chưa tồn tại.

#### **Ví dụ sử dụng:**
```shell
POST http://localhost:3000/save_score
Content-Type: application/json

{
    "url_video": "https://example.com/video1",
    "criteria": "Fluency",
    "score": 8,
    "note": "Excellent fluency"
}
```

**Response**:
```json
{
    "message": "Score saved successfully",
    "id": 1
}
```

---

### 3. **API: `/get_scores`**
- **Phương thức**: `GET`
- **Tham số yêu cầu**: `url_video`
- **Chức năng**:
  - Lấy danh sách tất cả các điểm số cho một video cụ thể.
  - Trả về các bản ghi sắp xếp theo `timestamp` (mới nhất trước).

#### **Ví dụ sử dụng:**
```shell
GET http://localhost:3000/get_scores?url_video=https://example.com/video1
```

**Response**:
```json
[
    {
        "id": 1,
        "url_video": "https://example.com/video1",
        "criteria": "Fluency",
        "score": 8,
        "note": "Excellent fluency",
        "timestamp": "2024-11-08T10:15:30"
    }
]
```

---

## **Gợi ý cải tiến**

1. **Bảo mật CORS**:
   - Hiện tại, bạn đang sử dụng `Access-Control-Allow-Origin: *`, điều này có thể không an toàn nếu ứng dụng của bạn triển khai trên môi trường sản xuất.
   - **Gợi ý**: Chỉ cho phép các domain cụ thể truy cập:
     ```python
     CORS(app, resources={
         r"/*": {
             "origins": ["https://yourdomain.com"]
         }
     })
     ```

2. **Xử lý kết nối CSDL**:
   - Bạn đang sử dụng `sqlite3.connect(DB_PATH)` trong từng route. Để **cải thiện hiệu suất**, bạn có thể sử dụng **connection pool** hoặc một context manager để đảm bảo đóng kết nối an toàn hơn.

3. **Xử lý lỗi**:
   - Bạn có thể tạo một **middleware** hoặc một hàm decorator để **bắt và xử lý các ngoại lệ** nhằm tránh lặp lại mã trong từng route.

4. **Tối ưu hóa database**:
   - Nếu `scores` hoặc `videos` có thể phát triển lớn dần, hãy cân nhắc **chuyển sang một CSDL mạnh hơn** như PostgreSQL hoặc MySQL.

5. **Bảo vệ API**:
   - Hiện tại, các API của bạn **không yêu cầu xác thực**. Nếu bạn muốn bảo vệ dữ liệu, hãy cân nhắc **thêm xác thực** bằng cách sử dụng JWT hoặc OAuth2.

---

## **Cách chạy ứng dụng**
```shell
python api_database_app.py
```

- Ứng dụng sẽ chạy tại `http://localhost:3000`.


## 4. GET TOÀN BỘ VIDEO 
I'll show you how to test the `/get_videos` endpoint using cURL, along with example input and output.

**cURL Command:**
```bash
curl -X GET http://localhost:3000/get_videos
```

**Expected Output:**
```json
[
    {
        "url_video": "https://drive.google.com/file/d/xxx1",
        "name_video": "Demo Video 1",
        "transcript": "This is the transcript for video 1",
        "criteria": "Criteria 1, Criteria 2, Criteria 3"
    },
    {
        "url_video": "https://drive.google.com/file/d/xxx2",
        "name_video": "Demo Video 2",
        "transcript": "This is the transcript for video 2",
        "criteria": "Criteria 1, Criteria 4, Criteria 5"
    }
]
```

**Error Response (if something goes wrong):**
```json
{
    "error": "Error message details"
}
```

**Notes:**
- This is a GET request, so no input body is required
- The response is an array of all videos in the database
- Each video object contains 4 fields: url_video, name_video, transcript, and criteria
- The results are sorted by name_video
- Status code will be:
  - 200: Success
  - 500: Server error

You can also test using a web browser by simply navigating to:
```
http://localhost:3000/get_videos
```